# Basic Trust Region Algorithm

In [1]:
using Optim

BasicTrustRegion will gather the constants of the method.

In [2]:
immutable BasicTrustRegion{T <: Real}
    η1:: T
    η2:: T
    γ1:: T
    γ2:: T
end

function BTRDefaults()
    return BasicTrustRegion(0.01,0.9,0.5,0.5)
end

BTRDefaults (generic function with 1 method)

We define a type to store the state at a given iteration.

In [6]:
type BTRState
    iter::Int
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64
    
    function BTRState()
        return new()
    end
end

In [7]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    # If the iteration is successful, update the iterate
    if (state.ρ >= b.η1)
        return true
    else
        return false
    end
end

acceptCandidate! (generic function with 1 method)

In [8]:
function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if (state.ρ >= b.η2)
        stepnorm = norm(state.step)
        state.Δ = min(1e20,max(4*stepnorm,state.Δ))
    elseif (state.ρ >= b.η1)
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

## Cauchy Step

Check the Cauchy Step value.

We have to minimize the model along the direction $-g$, i.e. $s = -\alpha g$.
$$
\min_{\alpha} m(-\alpha g) = -\alpha g^Tg + 0.5 \alpha^2 g^TH g \mbox{ s.t. } \| \alpha g \| \leq \Delta
$$

If $g^T H g = 0$, the problem is linear and the solution has to satisfy $\| s \| = \Delta$.
 
Moreover, if $\exists u$ such that $u^THu < 0$, then
$$
\lim_{\alpha \rightarrow \pm\infty} \alpha u^T H \alpha u
= \lim_{\alpha \rightarrow \pm\infty} \alpha^2 u^T H u = -\infty
$$
$u$ is said to be a negative curvature direction.

Thus, if $g^THg \leq 0$ the optimal solution is on the boundary: $\|s\| = \Delta$, or $\| \alpha g \| = \Delta$. Therefore, $\alpha = \Delta/\|g\|$ and
$$
s = -\frac{\Delta g}{\| g \|}.
$$

Consider now the case $g^T H g > 0$. Thus $m(-\alpha g)$ is a strictly convex function in $\alpha$ and the unconstrained minimizer is obtained by computing the zero of the model derivative:
$$
0 = \frac{d m(-\alpha g)}{d\alpha} = -\| g \|^2 + \alpha g^T H g,
$$
leading to
$$
\alpha = \frac{\| g \|^2}{g^T H g}
$$
and
$$
s = -\frac{\| g \|^2}{g^T H g} g.
$$

But $s$ has to be inside the trust-region. Thus we have
$$
s =
\begin{cases}
    -\frac{\| g \|^2}{g^T H g}g & \mbox{ if }\frac{\| g \|^3}{g^T H g} \leq \Delta \\
    -\frac{\Delta g}{\| g \|} & \mbox{ otherwise.}
\end{cases}
$$

In [10]:
function CauchyStep(g::Vector,H::Matrix,Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)
    if (q <= 0)
        τ = 1.0
    else
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end
    return -τ*g*Δ/normg
end

CauchyStep (generic function with 1 method)

## Basic Trust Region Algorithm

In [20]:
function btr(f::Function, g!::Function, H!::Function,
    x0::Vector, tol::Float64 = 1e-8, verbose::Bool = false)
    
    b = BTRDefaults()
    state = BTRState()
    state.iter = 0
    state.Δ = 1.0
    state.x = x0
    n=length(x0)

    tol2 = tol*tol
    
    state.g = zeros(n)
    H = zeros(n,n)
    
    fx = f(x0)
    g!(x0, state.g)
    H!(x0, H)
    
    nmax = 100000
    
    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while (dot(state.g,state.g) > tol2 && state.iter < nmax)
        # Compute the step by approximately minimize the model
        state.step = CauchyStep(state.g, H, state.Δ)
        state.xcand = state.x+state.step

        # Compute the actual reduction over the predicted reduction
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))

        if (acceptCandidate!(state, b))
            state.x = copy(state.xcand)
            g!(state.x, state.g)
            H!(state.x, H)
            fx = fcand
        end

        updateRadius!(state, b)
        state.iter += 1
    end
    
    return state
end

btr (generic function with 3 methods)

## Example

In [11]:
function rosenbrock(x::Vector)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

function rosenbrock_gradient!(x::Vector, storage::Vector)
    storage[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    storage[2] = 200.0 * (x[2] - x[1]^2)
end

function rosenbrock_hessian!(x::Vector, storage::Matrix)
    storage[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    storage[1, 2] = -400.0 * x[1]
    storage[2, 1] = -400.0 * x[1]
    storage[2, 2] = 200.0
end

rosenbrock_hessian! (generic function with 1 method)

In [21]:
state = btr(rosenbrock, rosenbrock_gradient!, rosenbrock_hessian!, [0,0])

BTRState(12143,[1.0,1.0],[1.0,1.0],[-7.6916e-9,-6.35603e-9],[-1.71328e-11,2.07329e-11],0.2462880346108132,0.999998819005788)

In [17]:
state

BTRState(1000,[0.925481,0.855962],[0.925481,0.855962],[0.0557776,-0.110653],[0.000265991,0.000133942],0.2462880346108132,0.9996072346055772)

"Trust-Region Methods", Introduction
$$
f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [22]:
using ForwardDiff

In [23]:
f(x::Vector) = -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4

f (generic function with 1 method)

In [24]:
g = x -> ForwardDiff.gradient(f, x);
H = x -> ForwardDiff.hessian(f, x)

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [39]:
function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

H! (generic function with 2 methods)

In [45]:
state = btr(f, g!, H!, [0,0],1e-8)

BTRState(100000,[2.30663,-0.332309],[2.30663,-0.332309],[1.63421e-8,-9.09602e-9],[-0.0,0.0],0.0,NaN)

In [46]:
state = btr(f, g!, H!, [0,0],1e-7)

BTRState(13,[2.30663,-0.332309],[2.30663,-0.332309],[1.63421e-8,-9.09602e-9],[2.07276e-9,3.72396e-9],1.9466028216398799,10.396850720617334)